# Filter Class


In [1]:
learning_rate = 0.01
beqa=[]

In [2]:
import numpy as np

class Filter:
    
    ### constructor takes array which will be filter
    def __init__(self, filterM):
        self.filter = np.array(filterM)
        self.n = self.filter.shape[0]
        self.m = self.filter.shape[1]
        
    ### returns filter
    def get_filter(self):
        return self.filter
    
    ### change filter by delta matrix
    def add_delta(self, delta):
        self.filter += delta

    ### set the value of i,j element of the filter 
    def set(self, i, j, val):
        self.filter[i][j] = val

    ### returns the value of i,j element
    def get(self, i, j):
        return self.filter[i][j]

    ### takes matrix and returns filtered matrix
    def go_filter(self, data):
        resultdata = np.zeros((data.shape[0] - self.n + 1, data.shape[1] - self.m + 1))
        for i in range(data.shape[0] - self.n + 1):
            for j in range(data.shape[1] - self.m + 1):
                resultdata[i][j] = np.sum(self.filter * data[i:i+self.n,j:j+self.m])
        return resultdata
    
    ### takes matrix and poolfilter dimension, returns max-pooled matrix
    def go_maxpool(self, data, n, m):
        resultdata = np.zeros((int(data.shape[0] / n) , int(data.shape[1] / m)))
        for i in range(0, data.shape[0], n):
            for j in range(0, data.shape[1], m):
                #print(i,j)
                resultdata[int(i/n)][int(j/m)] = np.max(data[i:i+n, j:j+m])
        return resultdata
    
    def update_filter_one_matrix(self, data, loss_data):
        updated_filter = np.zeros(self.filter.shape)
        updated_data = np.zeros(data.shape)
        for i in range(data.shape[0] + 1 - self.n):
            for j in range(data.shape[1] + 1 - self.m):
                # in summary resultdata[i][j] = self.filter * data[i:i+self.n, j : j+self.m]
                my_filter_part_data = data[i:i+self.n, j : j+self.m]
                my_data_part_data = updated_data[i:i+self.n, j : j+self.m]
                for x in range(my_filter_part_data.shape[0]):
                    for y in range(my_filter_part_data.shape[1]):
                        updated_filter[x][y] += loss_data[i][j]*my_filter_part_data[x][y]
                        # x derivative
                        my_data_part_data[x][y] +=loss_data[i][j]*self.filter[x][y]
                updated_data[i:i+self.n, j : j+self.m] = my_data_part_data
        self.filter =self.filter -  learning_rate*updated_filter
#         print(updated_filter)
        return updated_filter, updated_data



In [3]:
# my_filter = Filter(np.array([1,2],[3,4]))
filt = np.array([[1,2],[3,4]])
data = np.array([[1,2],[3,4]])
loss = np.array([[1,1],[1,1]])
my_filter = Filter(filt)
a,b = my_filter.update_filter_one_matrix(data,loss)
print(type(filt))

# filt-= a*learning_rate


<class 'numpy.ndarray'>


In [4]:
filt= filt - a*learning_rate

# Layer Class

In [5]:

class Layer:
    ### constructor takes the list of filters 
    def __init__(self, num_filters):
#         self.updated_filters = [np.zeros(matr.shape) for matr in self.filters]
#         self.filters = [np.random.randn(2,2) for x in range(num_filters)]
        self.filters = []
        self.num_filters = num_filters
        self.matrixes = []
        self.preMatrixes = []
        self.n = 2
        self.m = 2
    
    ### returns the i-th filter
    def get_filter(self, i):
        return self.filters[i]
    
    ### return the list of filters
    def get_filters(self):
        return self.filters
    
    ### generates list of matrixes, from previous layer's matrixes and it's filters
    def gen_matrixes(self, preMatrixes):
        if len(self.filters) == 0:
            for x in range(len(preMatrixes)):
                each_filters = [Filter(np.random.randn(2,2)) for _ in range(self.num_filters)]
                self.filters.append(each_filters)
                
        self.preMatrixes = preMatrixes
#         print(len(self.preMatrixes))
        my_matrixes = [np.zeros((self.preMatrixes[0].shape[0] - self.n + 1, self.preMatrixes[0].shape[1] - self.m + 1)) for _ in range(self.num_filters)]
#         print("aeeeee")
#         print(my_matrixes)
        for x in range(len(preMatrixes)):
            mat = preMatrixes[x]
            temp = 0
            for filt in self.filters[x]:
                my_matrixes[temp] += filt.go_filter(mat)
                temp+=1
        self.matrixes = my_matrixes
            
        
    ### returns i-th matrix of our matrixs' list
    def get_matrix(self, i):
        return self.matrix[i]
    
    ### returns list of matrixes
    def get_matrixes(self):
        return self.matrixes
    
    def layer_back_forward(self, loss_data_list):


        pre_matrix_list = self.preMatrixes
        updated_pre_matrix = [np.zeros(matr.shape) for matr in pre_matrix_list]
        
        updated_filters = []

        for x in range(len(self.filters)):
            each_filters = [np.zeros(matr.get_filter().shape) for matr in self.filters[x]]
            updated_filters.append(each_filters)
#         print(self.filters)
        for x in range(len(self.filters)):
            filt = self.filters[x]
            for index in range(len(filt)):
                cur_filt = filt[index]
                cur_loss_data = loss_data_list[index]
                for y in range(len(pre_matrix_list)):
                    mat = pre_matrix_list[y]
                    updated_filter, updated_data = cur_filt.update_filter_one_matrix(mat, cur_loss_data)
                    updated_filters[x][index]+=updated_filter
                    updated_pre_matrix[y]+=updated_data
#         self.filters -= learning_rate*updated_filters
#         for x in range(len(self.filters)):
#             filt_list = self.filters[x]
#             for y in range(len(filt_list)):
#                 self.filters[x][y]-=learning_rate*updated_filters[x][y]
                
        
        
        return updated_pre_matrix
           
    


# CNN Class

In [6]:
class CNN:
    ### constructor takes the starting matrix and filters list for each layer
    ### layerFilters is 4D list
    ### a = CNN(ar, )
    ## update.
    ## layerFilters listshi mewera listi filtrebisa layerebis mixedvit
    ## exla shignit ubralod ricxvs chavwer tu ramdeni filtri unda mas.
    def __init__(self, stMatrix, layerFilters):
        self.stMatrix = stMatrix
        self.layers = []
        for layerFilter in layerFilters:
#             print(layerFilter)
            self.layers.append(Layer(layerFilter))
    
    ### returns all layer objects 
    def get_layers(self):
        return self.layers
    def set_x(self,X):
        self.stMatrix = X
    
    ### returns i-th layer
    def get_layer(self, i):
        return self.layers[i]
        
    ### returns flattened np-array for fully connection NL
    def forward_prop(self):
        matrixList = [self.stMatrix]
        for layer in self.layers:
            layer.gen_matrixes(matrixList)
            matrixList = layer.get_matrixes()
        return self.flatten(matrixList)
            
    ### static method, flattens 3D array
    def flatten(self, matrixList):
#         print(matrixList)
#         print("chemi listukaa")
#         print(len(matrixList))
        lst = []
        for matrix in matrixList:
            #print(matrix,matrix.flatten())
            lst+=(matrix.flatten().tolist())
        return lst
    
    def unflatten(self, flatten_matrix):
        my_matrix = self.layers[len(self.layers)-1].get_matrixes()
#         print(self.layers[0])
        my_loss_matrix = [np.zeros(tmp.shape) for tmp in my_matrix]
        temp = 0
        for index in range(len(my_loss_matrix)):
            mat = my_loss_matrix[index]
            for x in range(mat.shape[0]):
                for y in range(mat.shape[1]):
#                     print(my_loss_matrix[index][x][y])
                    my_loss_matrix[index][x][y] = flatten_matrix[0][temp]
                    temp+=1
        return my_loss_matrix
        
    def backward(self, delta_x):
#         print(len(self.layers))
#         pass
#         print(delta_x)
        new_delta_x = delta_x
        for layer in reversed(self.layers):
            #WINEBIS listi da loss_datas listi
            new_delta_x =layer.layer_back_forward(new_delta_x)
        
            
        
    

In [7]:
# cnn = CNN(np.array[np.random.randn(4,4)],[[Filter([[1,1],[1,1]])]])
# print(cnn.forward_prop())
# #CNN.flatten([np.array([[1,2],[3,4]]),np.array([[1,2,3],[3,4,5]])])



# Dance NL Class

In [8]:
class Dense:
    
    ### constructor takes parameter's array(flattened array) and answer array(1x10)
    def __init__(self, X, Y):
        self.i=0
        self.X = X
        self.Y = Y
#         print(X.shape)
#         print(Y.shape)
#         print(self.theta0.shape)
#         print(self.theta1.shape)
       
        # just return e^(-z)
    def my_exp(self,z):
        return np.exp(-z)
    
    def set_y(self, Y):
        self.Y=Y
    
    def set_matrix(self,mat):
        if self.X is None:
            self.W0 = np.random.randn(mat.size, 64)
            self.W1 = np.random.randn(64, 10)
        self.X = mat
        
        
        # our zigmoid function
    def zigmoid_function(self,z):
        return 1/(1 + self.my_exp(z))
    
    def costfn_class(self, X, y):
#         X = self.zigmoid_function(X)
        error= -1*y*np.log(X) + (y-1)*np.log(1-X)
#         my_sum = np.sum(abs(error))
        return error
    
    def cross_entropy(self, predictions, targets, epsilon=1e-10):
#         predictions = np.clip(predictions, epsilon, 1. - epsilon)
#         N = predictions.shape[0]
#         ce_loss = targets * np.log(predictions + 1e-5)
#         return ce_loss    
        predictions = np.clip(predictions, epsilon, 1. - epsilon)
        N = predictions.shape[0]
        ce = -1*targets*np.log(predictions+1e-9) + (targets - 1)*np.log(1-predictions + 1e-9)
        return ce
    
    
        # ferivative of our zigmoid function
    def derivative_zigmoid(self,z):
        return self.my_exp(z) / np.power((1 + self.my_exp(z)), 2)
    
    # our fucntion doing feed forward alghoritm.
    def feed_forward(self,X, W0, W1):
        h0 = np.dot(X, W0)  # our first matrix after using first hidden layer W0
    #     h0 = zigmoid_function(h0)  # our first matrix using zigmoid.
        
        h3 = np.dot(h0, W1) # our result after using second hidden layer W1
        h1 = self.zigmoid_function(h3) # our result matrix after using zigmoid.
        return h0, h1 # return result.
    
    # compute our error which is h1-y but we also take into consideration the zigmoid function.
    def my_error(self,h1):
        error = h1 - self.Y
        return error

    def costfn(self,h1):
        error = self.my_error(h1)
        my_sum = np.sum(abs(error))
        return my_sum/len(error)


    def backward(self,X, W0, W1, h1, h0, alfa_rate):
        # firstly lets compute error h1-y
        error = self.my_error(h1)
        bla_W1 = W1
        # first lets update W1 which is far easier W1 = (h0)T *(h1 - Y)
        transpose_h0 = np.transpose(h0)
        error = error * self.derivative_zigmoid(h1)

        # now lets update W0 too which is bit complex.
        # W0 = XT.(ERROR).W1T
        transpose_X = np.transpose(X)
        transpose_W1 = np.transpose(bla_W1)


        first_step = np.dot(error, transpose_W1)
        second_step = np.dot(transpose_X, first_step)
        delta_x = np.dot(error, np.transpose(np.dot(self.W0, self.W1)))
        W0-=second_step*alfa_rate
        W1 -= np.dot (transpose_h0, error)*alfa_rate
#         print(W0)
#         print(W1)
        return delta_x
    

    def predict(self):
        
        h0, h1 = self.feed_forward(self.X, self.W0, self.W1)
        delta_x = self.backward(self.X, self.W0, self.W1, h1, h0, 0.00001)
#         print(self.cross_entropy(h1,Y))
        return delta_x, h1
#         return None, None

In [9]:
# from PIL import Image
# im = Image.open('../Letters/'+'ე'+'/'+'ე10.jpg')
# im2arr = np.array(im)

In [10]:
############## LETS INITIALIZE OUR LAYERS AND FILTERS
# X = im2arr
# Y = [[0,1,0,0,0,0,0,0,0,0]]
# Y=np.array(Y)
learning_rate = 0.000007
# here we will initialize first layer filters
# first_layer_filter = Filter(np.random.randn(5,5))
first_layer_filters = []
first_layer_filters.append(1)

# here we will initialize second layer filters
# second_layer_filter = Filter(np.random.randn(3,3))
second_layer_filters = []
second_layer_filters.append(1)

# here we will initialize second layer filters
filter_array = []
filter_array.append(2)
filter_array.append(2)

In [11]:
##### lets initialize CNN class
# cnn = CNN(X,filter_array)
# dense_class = Dense(None,Y)
cnn = CNN(None,filter_array)
dense_class = Dense(None,None)

In [12]:
def predict(cnn,dense_class):
    ### this method goes cnn part till flattened array.#         self.filters -= learning_rate*updated_filters

    flatten_array = np.array(cnn.forward_prop())
#     print(flatten_array.max())
    flatten_array = np.reshape(np.array(flatten_array), (flatten_array.shape[0],1))
    ###now lets start with dense class to work with neural network.
    flatten_array = np.transpose(flatten_array)
#     dense_class = Dense(np.transpose(flatten_array),Y)
    dense_class.set_matrix(flatten_array)
    delta_x, h1 = dense_class.predict()
# #     print(delta_x.max())
#     #listi maq matricebis
    delta_x = cnn.unflatten(delta_x)
#     #backward!!!!!!!
#     # now we have to write flatten back to matrixes.
#     ####
    cnn.backward(delta_x)
    return h1

In [15]:
from PIL import Image
im = Image.open('../Letters/'+'ე'+'/'+'ე10.jpg')
im2arr = np.array(im)
X = im2arr
Y = [[0,1,0,0,0,0,0,0,0,0]]
Y=np.array(Y)
def go_forest(X,Y):
    cnn.set_x(X)
    dense_class.set_y(Y)
    for _ in range(15):
        h1 = predict(cnn,dense_class)
    print(h1)

In [16]:
go_forest(X,Y)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:15: RuntimeWarning: overflow encountered in exp
  from ipykernel import kernelapp as app


[[0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]]
